In [1]:
import pandas as pd
# ^^^ pyforest auto-imports - don't write above this line
#importing the required libraries 
import pyforest

In [2]:
#loading data from CSSEGISandData COVID-19 (John Hopkins)

confirmed = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv')
deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Deaths.csv')
recoveries = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Recovered.csv')

In [3]:
confirmed.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20
0,NaN,Thailand,15.0000,101.0000,2,3,5,7,8,8,...,43,43,43,47,48,50,50,50,53,59
1,NaN,Japan,36.0000,138.0000,2,1,2,2,4,4,...,274,293,331,360,420,461,502,511,581,639
2,NaN,Singapore,1.2833,103.8333,0,1,3,3,4,5,...,108,110,110,117,130,138,150,150,160,178
3,NaN,Nepal,28.1667,84.2500,0,0,0,1,1,1,...,1,1,1,1,1,1,1,1,1,1
4,NaN,Malaysia,2.5000,112.5000,0,0,0,3,4,4,...,29,36,50,50,83,93,99,117,129,149


In [4]:
#Dropping the  'Province/State' column from all the three datasets as its completely null
confirmed.drop(labels = 'Province/State', axis = 1, inplace = True)
deaths.drop(labels = 'Province/State', axis = 1, inplace = True)
recoveries.drop(labels = 'Province/State', axis = 1, inplace = True)

In [5]:
confirmed.isnull().sum()

Country/Region    0
Lat               0
Long              0
1/22/20           0
1/23/20           0
1/24/20           0
1/25/20           0
1/26/20           0
1/27/20           0
1/28/20           0
1/29/20           0
1/30/20           0
1/31/20           0
2/1/20            0
2/2/20            0
2/3/20            0
2/4/20            0
2/5/20            0
2/6/20            0
2/7/20            0
2/8/20            0
2/9/20            0
2/10/20           0
2/11/20           0
2/12/20           0
2/13/20           0
2/14/20           0
2/15/20           0
2/16/20           0
2/17/20           0
2/18/20           0
2/19/20           0
2/20/20           0
2/21/20           0
2/22/20           0
2/23/20           0
2/24/20           0
2/25/20           0
2/26/20           0
2/27/20           0
2/28/20           0
2/29/20           0
3/1/20            0
3/2/20            0
3/3/20            0
3/4/20            0
3/5/20            0
3/6/20            0
3/7/20            0
3/8/20            0


In [6]:
#Converting the date labels to rows

def cleandata(df_raw):
    df_cleaned = df_raw.melt(id_vars = ['Country/Region', 'Lat', 'Long'], value_name = 'Cases', var_name = 'Date')
    df_cleaned = df_cleaned.set_index(['Country/Region','Date'])
    return df_cleaned

In [7]:
confirmed_tot = cleandata(confirmed)
deaths_tot = cleandata(deaths)
recoveries_tot = cleandata(recoveries)

In [8]:
confirmed_tot.head(2)

,,Lat,Long,Cases
Country/Region,Date,,,
Thailand,1/22/20,15.0,101.0,2
Japan,1/22/20,36.0,138.0,2


In [9]:
def countrydata(df_cleaned,oldname,newname):
    df_country=df_cleaned.groupby(['Country/Region','Date'])['Cases'].sum().reset_index()
    df_country=df_country.set_index(['Country/Region','Date'])
    df_country.index=df_country.index.set_levels([df_country.index.levels[0], pd.to_datetime(df_country.index.levels[1])])
    df_country=df_country.sort_values(['Country/Region','Date'],ascending=True)
    df_country=df_country.rename(columns={oldname:newname})
    return df_country

In [10]:
ConfirmedCasesCountry=countrydata(confirmed_tot,'Cases','Total Confirmed Cases')
ConfirmedDeathsCountry=countrydata(deaths_tot,'Cases','Total Deaths')
ConfirmedRecoveriesCountry=countrydata(recoveries_tot,'Cases','Total Recoveries')

In [11]:
ConfirmedCasesCountry

Total Confirmed Cases
Country/Region Date                             
Afghanistan    2020-01-22                      0
               2020-01-23                      0
               2020-01-24                      0
               2020-01-25                      0
               2020-01-26                      0
...                                          ...
Vietnam        2020-03-07                     18
               2020-03-08                     30
               2020-03-09                     30
               2020-03-10                     31
               2020-03-11                     38

[5700 rows x 1 columns]

In [12]:
#Getting daily data from cummulated sum values in Cases. 
def dailydata(dfcountry,oldname,newname):
    dfcountrydaily=dfcountry.groupby(level=0).diff().fillna(0)
    dfcountrydaily=dfcountrydaily.rename(columns={oldname:newname})
    return dfcountrydaily

In [13]:
CasesCountry=dailydata(ConfirmedCasesCountry,'Total Confirmed Cases','Daily Cases')
DeathsCountry=dailydata(ConfirmedDeathsCountry,'Total Deaths','Daily Deaths')
RecoveriesCountry=dailydata(ConfirmedRecoveriesCountry,'Total Recoveries','Daily recoveries')

In [14]:
CasesCountry.head()

Daily Cases
Country/Region Date                   
Afghanistan    2020-01-22          0.0
               2020-01-23          0.0
               2020-01-24          0.0
               2020-01-25          0.0
               2020-01-26          0.0

In [15]:
#Merging all the datasets to create daily and Cummulated sum values
covid_19 = pd.merge(ConfirmedCasesCountry,CasesCountry,left_index=True,right_index=True )
covid_19 = pd.merge(covid_19,DeathsCountry,left_index=True,right_index=True )
covid_19=pd.merge(covid_19,ConfirmedDeathsCountry,how='left',left_index=True,right_index=True)
covid_19=pd.merge(covid_19,RecoveriesCountry,how='left',left_index=True,right_index=True)
covid_19=pd.merge(covid_19,ConfirmedRecoveriesCountry,how='left',left_index=True,right_index=True)

In [16]:
covid_19.head()

Total Confirmed Cases  Daily Cases  Daily Deaths  \
Country/Region Date                                                           
Afghanistan    2020-01-22                      0          0.0           0.0   
               2020-01-23                      0          0.0           0.0   
               2020-01-24                      0          0.0           0.0   
               2020-01-25                      0          0.0           0.0   
               2020-01-26                      0          0.0           0.0   

                           Total Deaths  Daily recoveries  Total Recoveries  
Country/Region Date                                                          
Afghanistan    2020-01-22             0               0.0                 0  
               2020-01-23             0               0.0                 0  
               2020-01-24             0               0.0                 0  
               2020-01-25             0               0.0                 0  
               2020-01-26             0               0.0                 0

In [19]:
#Extracting data to CSV file for tableau visualization
covid_19.reset_index()
covid_19.to_csv(r"C:\Users\Chintu\Downloads\covid-19.csv")

In [20]:
covid_19.head()

Total Confirmed Cases  Daily Cases  Daily Deaths  \
Country/Region Date                                                           
Afghanistan    2020-01-22                      0          0.0           0.0   
               2020-01-23                      0          0.0           0.0   
               2020-01-24                      0          0.0           0.0   
               2020-01-25                      0          0.0           0.0   
               2020-01-26                      0          0.0           0.0   

                           Total Deaths  Daily recoveries  Total Recoveries  
Country/Region Date                                                          
Afghanistan    2020-01-22             0               0.0                 0  
               2020-01-23             0               0.0                 0  
               2020-01-24             0               0.0                 0  
               2020-01-25             0               0.0                 0  
               2020-01-26             0               0.0                 0

In [22]:
covid_19_daily = covid_19[['Daily Cases', 'Daily Deaths', 'Daily recoveries']]

In [23]:
covid_19_daily.head(2)

Daily Cases  Daily Deaths  Daily recoveries
Country/Region Date                                                   
Afghanistan    2020-01-22          0.0           0.0               0.0
               2020-01-23          0.0           0.0               0.0